In [1]:
# Install dependencies

%pip install -qU langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


Set the environment variables. We need an API key and an LLM server, at minimum.

In [3]:
import os

# We will add the api key as an environment variable. 
# You can set this value directrly form the shell,
# instead of explicitly setting it on code.
os.environ["OPENAI_API_KEY"] = "..."
# We need a custom endpoint, as we will be calling Verde's LLM
API_ENDPOINT = "https://llm1.cyverse.ai/v1"

The first step is to create a `Chat` object. This will allow us to call it through langchain and later on using `LCEL`

In [9]:
from langchain_openai import ChatOpenAI 		# We use the OpenAI protocol, but are using another provider (Verde)

# We will connect to Mistral Instruct v0.3 through Verde
# Notice how we need to specify the API endpoint
model = ChatOpenAI(model="Mistral-7B-Instruct-v0.3", base_url=API_ENDPOINT)

# Do a test call
from pprint import pprint
response = model.invoke("Hello, who are you?")

pprint(response)

AIMessage(content=" Hello! I am a model of an AI assistant. I'm designed to help answer questions, provide information, and assist with various tasks. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 9, 'total_tokens': 46}, 'model_name': 'mistralai/Mistral-7B-Instruct-v0.3', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6297cefc-1c73-4de9-b059-2aebbedfe2f9-0', usage_metadata={'input_tokens': 9, 'output_tokens': 37, 'total_tokens': 46})


Chat applications build upon the message history paradigm. There are three types of messages:

- System
- Human
- AI

The message history represents the conversation between the LLM and the user. The `System` message is optional. When present, it is usually used to define the _persona_ of the LLM. For example, establishing a role, defining a task, etc.

Then, the history is composed of alternating `Human` - `AI` messages.

In [14]:
# Let's build a conversation history

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a software developer, expert in the use of LLMs and langchain. Help the user answering the questions using didactic examples"),										# Use the system to establish the task of the LLM
    HumanMessage(content="What is Mistral and why would I need to use it"),		# This is the first "Human" message
]

# Instead of invoking with a string, pass the whole message history
response = model.invoke(messages)

# Add the response to the history
messages.append(response)

# Let's peek into the response
print(response.content)


 Mistral is an open-source large language model (LLM) developed by Mistral AI, a French AI company. It is designed to understand and generate human-like text in French and English.

You might need to use Mistral if you require a high-quality language model for various natural language processing (NLP) tasks, such as:

1. Text generation: Creating coherent and contextually relevant text, such as articles, summaries, or responses to user queries.
2. Translation: Translating text from French to English (or vice versa) with high accuracy and fluency.
3. Question answering: Answering questions based on a given context or set of documents, providing accurate and concise responses.
4. Text summarization: Condensing lengthy documents into summaries that retain the key points and maintain coherence.
5. Chatbot development: Building conversational agents that can engage in human-like interactions, providing assistance, information, or entertainment.

Compared to other popular LLMs, Mistral has s

Observe how the return value of the LLM model is an `AIMessage` instance. We add that response to the message history, which will be used for a follow up question

In [15]:
pprint(messages)

[SystemMessage(content='You are a software developer, expert in the use of LLMs and langchain. Help the user answering the questions using didactic examples'),
 HumanMessage(content='What is Mistral and why would I need to use it'),
 AIMessage(content=" Mistral is an open-source large language model (LLM) developed by Mistral AI, a French AI company. It is designed to understand and generate human-like text in French and English.\n\nYou might need to use Mistral if you require a high-quality language model for various natural language processing (NLP) tasks, such as:\n\n1. Text generation: Creating coherent and contextually relevant text, such as articles, summaries, or responses to user queries.\n2. Translation: Translating text from French to English (or vice versa) with high accuracy and fluency.\n3. Question answering: Answering questions based on a given context or set of documents, providing accurate and concise responses.\n4. Text summarization: Condensing lengthy documents into

Now, let's use a follow up query that relies on the coneversation history.

In [16]:
messages.append(HumanMessage("Give me a short example of a python script that calls the aforementioned LLM"))

response = model.invoke(messages)
messages.append(response)

print(response.content)

 To provide a simple example of using Mistral in Python, first, you'll need to install the Mistral AI SDK. You can find the installation instructions here: https://github.com/mistralai/mistral-sdk-python

Assuming you have the SDK installed, here's a short example of a Python script that uses the Mistral model for generating text:

```python
from mistral.applications.text_generation import TextGenerationApplication

# Initialize the TextGenerationApplication with the Mistral model
app = TextGenerationApplication()

# Set the input prompt
prompt = "What should I do if I have too much garbage at home?"

# Generate text based on the prompt
response = app.predict(prompt=prompt, max_tokens=64)

# Print the generated text
print(response)
```

In this example, we're using the TextGenerationApplication from the Mistral SDK to generate a response to a user question. The `prompt` variable contains the question, and the `max_tokens` parameter specifies the maximum number of tokens (words) to gene

Notice how we didn't explicity refered to Mistral, instead it was picked up from the message history.

Writting messages can get really tedious. We make use of message templates to simplify tasks by parameterizing the values.

We will use a template to create a document summarizer built on top of the LLM

In [23]:
# Import the chat template
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
	[("system", "Summarize the document provided by the user into a succint, at most three-sentence statement"),
	("user", "{text}")]
)

document = """
The Congress shall have Power To lay and collect Taxes, Duties, Imposts and Excises, to pay the Debts and provide for the common Defence and general Welfare of the United States; but all Duties, Imposts and Excises shall be uniform throughout the United States;
To borrow Money on the credit of the United States;
To regulate Commerce with foreign Nations, and among the several States, and with the Indian Tribes;
To establish an uniform Rule of Naturalization, and uniform Laws on the subject of Bankruptcies throughout the United States;
To coin Money, regulate the Value thereof, and of foreign Coin, and fix the Standard of Weights and Measures;
To provide for the Punishment of counterfeiting the Securities and current Coin of the United States;
To establish Post Offices and post Roads;
To promote the Progress of Science and useful Arts, by securing for limited Times to Authors and Inventors the exclusive Right to their respective Writings and Discoveries;
To constitute Tribunals inferior to the supreme Court;
To define and punish Piracies and Felonies committed on the high Seas, and Offences against the Law of Nations;
To declare War, grant Letters of Marque and Reprisal, and make Rules concerning Captures on Land and Water;
To raise and support Armies, but no Appropriation of Money to that Use shall be for a longer Term than two Years;
To provide and maintain a Navy;
To make Rules for the Government and Regulation of the land and naval Forces;
To provide for calling forth the Militia to execute the Laws of the Union, suppress Insurrections and repel Invasions;
To provide for organizing, arming, and disciplining, the Militia, and for governing such Part of them as may be employed in the Service of the United States, reserving to the States respectively, the Appointment of the Officers, and the Authority of training the Militia according to the discipline prescribed by Congress;
To exercise exclusive Legislation in all Cases whatsoever, over such District (not exceeding ten Miles square) as may, by Cession of particular States, and the Acceptance of Congress, become the Seat of the Government of the United States, and to exercise like Authority over all Places purchased by the Consent of the Legislature of the State in which the Same shall be, for the Erection of Forts, Magazines, Arsenals, dock-Yards, and other needful Buildings;—And
To make all Laws which shall be necessary and proper for carrying into Execution the foregoing Powers, and all other Powers vested by this Constitution in the Government of the United States, or in any Department or Officer thereof.
"""

prompt = prompt_template.invoke({"text": document})
pprint(prompt)

print(model.invoke(prompt).content)

ChatPromptValue(messages=[SystemMessage(content='Summarize the document provided by the user into a succint, at most three-sentence statement'), HumanMessage(content='\nThe Congress shall have Power To lay and collect Taxes, Duties, Imposts and Excises, to pay the Debts and provide for the common Defence and general Welfare of the United States; but all Duties, Imposts and Excises shall be uniform throughout the United States;\nTo borrow Money on the credit of the United States;\nTo regulate Commerce with foreign Nations, and among the several States, and with the Indian Tribes;\nTo establish an uniform Rule of Naturalization, and uniform Laws on the subject of Bankruptcies throughout the United States;\nTo coin Money, regulate the Value thereof, and of foreign Coin, and fix the Standard of Weights and Measures;\nTo provide for the Punishment of counterfeiting the Securities and current Coin of the United States;\nTo establish Post Offices and post Roads;\nTo promote the Progress of Sc